In [1]:
# Instal dependency
!pip install open_clip_torch

In [2]:
# Import module
import os
import open_clip
import glob
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

# Parse data path

In [6]:
keyframes_dir = '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/L00_reduced'
# all_keyframe_paths = dict()
# for part in sorted(os.listdir(keyframes_dir)):
#     data_part = part.split('_')[-1] # L01, L02 for ex
#     all_keyframe_paths[data_part] =  dict()

# for data_part in sorted(all_keyframe_paths.keys()):
#     data_part_path = f'{keyframes_dir}/{data_part}'
#     video_dirs = sorted(os.listdir(data_part_path))
#     video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
#     for video_id, video_dir in zip(video_ids, video_dirs):
#         keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.webp'))
#         all_keyframe_paths[data_part][video_id] = keyframe_paths

# Model

In [4]:
##### Load Model #####
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', device=device, pretrained='datacomp_xl_s13b_b90k')

cpu


open_clip_pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/57/8d/578ddc43cf2804098ab8806beffdc2c2b91b0e5fe4e88c534bb0b996f11c78e5/6509f07e6fc0da68f8e1ee881bf90803f0b053d2f7ed2013cc7c3a49ac4dd3db?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27open_clip_pytorch_model.bin%3B+filename%3D%22open_clip_pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1728686826&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODY4NjgyNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy81Ny84ZC81NzhkZGM0M2NmMjgwNDA5OGFiODgwNmJlZmZkYzJjMmI5MWIwZTVmZTRlODhjNTM0YmIwYjk5NmYxMWM3OGU1LzY1MDlmMDdlNmZjMGRhNjhmOGUxZWU4ODFiZjkwODAzZjBiMDUzZDJmN2VkMjAxM2NjN2MzYTQ5YWM0ZGQzZGI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=c-3N%7EsBhvHh3VVOUKzA9tVuHlw64TfEiyxdPR-HgB5h5D2nduUZ4Q4hLMLSYan2WjT%7EDYPGZyGpp4y3Ubzr3WF6Mq1huYfVFkvqtP07oMwI2-8bspTpr4c8ZmVqoVSteF9%7EpK18UkY7H1CBzxMuh%7ElQyK1NFjE

open_clip_pytorch_model.bin:  29%|##8       | 493M/1.71G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/57/8d/578ddc43cf2804098ab8806beffdc2c2b91b0e5fe4e88c534bb0b996f11c78e5/6509f07e6fc0da68f8e1ee881bf90803f0b053d2f7ed2013cc7c3a49ac4dd3db?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27open_clip_pytorch_model.bin%3B+filename%3D%22open_clip_pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1728686826&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODY4NjgyNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy81Ny84ZC81NzhkZGM0M2NmMjgwNDA5OGFiODgwNmJlZmZkYzJjMmI5MWIwZTVmZTRlODhjNTM0YmIwYjk5NmYxMWM3OGU1LzY1MDlmMDdlNmZjMGRhNjhmOGUxZWU4ODFiZjkwODAzZjBiMDUzZDJmN2VkMjAxM2NjN2MzYTQ5YWM0ZGQzZGI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=c-3N%7EsBhvHh3VVOUKzA9tVuHlw64TfEiyxdPR-HgB5h5D2nduUZ4Q4hLMLSYan2WjT%7EDYPGZyGpp4y3Ubzr3WF6Mq1huYfVFkvqtP07oMwI2-8bspTpr4c8ZmVqoVSteF9%7EpK18UkY7H1CBzxMuh%7ElQyK1NFjE

...

Error while downloading from https://cdn-lfs.hf.co/repos/57/8d/578ddc43cf2804098ab8806beffdc2c2b91b0e5fe4e88c534bb0b996f11c78e5/6509f07e6fc0da68f8e1ee881bf90803f0b053d2f7ed2013cc7c3a49ac4dd3db?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27open_clip_pytorch_model.bin%3B+filename%3D%22open_clip_pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1728686826&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODY4NjgyNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy81Ny84ZC81NzhkZGM0M2NmMjgwNDA5OGFiODgwNmJlZmZkYzJjMmI5MWIwZTVmZTRlODhjNTM0YmIwYjk5NmYxMWM3OGU1LzY1MDlmMDdlNmZjMGRhNjhmOGUxZWU4ODFiZjkwODAzZjBiMDUzZDJmN2VkMjAxM2NjN2MzYTQ5YWM0ZGQzZGI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=c-3N%7EsBhvHh3VVOUKzA9tVuHlw64TfEiyxdPR-HgB5h5D2nduUZ4Q4hLMLSYan2WjT%7EDYPGZyGpp4y3Ubzr3WF6Mq1huYfVFkvqtP07oMwI2-8bspTpr4c8ZmVqoVSteF9%7EpK18UkY7H1CBzxMuh%7ElQyK1NFjE

open_clip_pytorch_model.bin:  67%|######6   | 1.14G/1.71G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/57/8d/578ddc43cf2804098ab8806beffdc2c2b91b0e5fe4e88c534bb0b996f11c78e5/6509f07e6fc0da68f8e1ee881bf90803f0b053d2f7ed2013cc7c3a49ac4dd3db?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27open_clip_pytorch_model.bin%3B+filename%3D%22open_clip_pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1728686826&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODY4NjgyNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy81Ny84ZC81NzhkZGM0M2NmMjgwNDA5OGFiODgwNmJlZmZkYzJjMmI5MWIwZTVmZTRlODhjNTM0YmIwYjk5NmYxMWM3OGU1LzY1MDlmMDdlNmZjMGRhNjhmOGUxZWU4ODFiZjkwODAzZjBiMDUzZDJmN2VkMjAxM2NjN2MzYTQ5YWM0ZGQzZGI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=c-3N%7EsBhvHh3VVOUKzA9tVuHlw64TfEiyxdPR-HgB5h5D2nduUZ4Q4hLMLSYan2WjT%7EDYPGZyGpp4y3Ubzr3WF6Mq1huYfVFkvqtP07oMwI2-8bspTpr4c8ZmVqoVSteF9%7EpK18UkY7H1CBzxMuh%7ElQyK1NFjE

open_clip_pytorch_model.bin:  84%|########3 | 1.44G/1.71G [00:00<?, ?B/s]

/opt/anaconda3/lib/python3.12/site-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map

In [7]:
bs = 4
save_dir = './CLIPv2_features'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

for key, video_keyframe_paths in tqdm(all_keyframe_paths.items()):
    video_ids = sorted(video_keyframe_paths.keys())
    
    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))
    
    for video_id in tqdm(video_ids):
        video_feats = []
        video_keyframe_path = video_keyframe_paths[video_id]
        for i in range(0, len(video_keyframe_path), bs):
            # Support batchsize inferencing
            images = []
            image_paths = video_keyframe_path[i:i+bs]
            for image_path in image_paths:
                image = preprocess(Image.open(image_path)).unsqueeze(0)
                images.append(image)
            images = torch.cat(images).to(device)

            with torch.no_grad(), torch.cuda.amp.autocast():
                image_feats = model.encode_image(images)
            image_feats /= image_feats.norm(dim=-1, keepdim=True)

            for b in range(image_feats.shape[0]):
                video_feats.append(image_feats[b].detach().cpu().numpy().astype(np.float32).flatten())
        
        np.save(f'{save_dir}/{key}/{video_id}.npy', video_feats)

0it [00:00, ?it/s]
